Importation of modules

In [1]:
import os
import glob
import numpy as np
import xarray as xr
from obspy.signal.filter import bandpass

from tools_pcc import *
from stack_functions_new import *

Paths

In [2]:
user = "ipgp"

if user == "ipgp" :
    path1 = "/media/hugonnet/DISKENOIR/fichiers-nc/"
    path2 = "/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/" # storage of correlograms for cross-correlation
    path3 = "/media/hugonnet/DISKENOIR/Correlograms/Auto-correlation/" # storage of correlograms for auto-correlation
    path4 = "/home/hugonnet/Documents/DAS-Lucie/Codes/Sergi-ts-PWS/src/" # location of ts_pws file

elif user == "mba" :
    path1 = "/Volumes/DISKENOIR/fichiers-nc/"
    path2 = "/Volumes/DISKENOIR/Correlograms/Cross-correlation/" # storage of correlograms for cross-correlation
    path3 = "/Volumes/DISKENOIR/Correlograms/Auto-correlation/" # storage of correlograms for auto-correlation
    path4 = "/Users/lucie/Dropbox/MINES_3A/Stage-IPGP/DAS/Codes/Sergi-ts-PWS/src/" # location of ts_pws file

elif user == "eleonore" :
    path1 = '/Users/stutz/DAS/DAS_STROMBOLI/2022_STROMBOLI_MANIP/SILIXA_iDAS/DATA_VLP_FEBUS/'

In [3]:
files = glob.glob(path1 + '*.nc')
size_files = len(files)

In [4]:
files.sort
files

['/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_00-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_01-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_02-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_03-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_04-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_05-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_06-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_07-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_08-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_09-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fichiers-nc/SR_FEBUS_STROMBOLI_2022-10-25_10-10-20_UTC.nc',
 '/media/hugonnet/DISKENOIR/fich

In [5]:
len(files)

242

Definition of the parameters

In [4]:
corr_type = 'phase-cross'   # can be 'phase-cross' (cross-correlation), 'phase-auto' (auto-correlation), 'normal', or 'onebit'

# define variable for correlation
window = 300 			# duration of time window in second
lag = window/2 			# shift of starting time on the next iteration
time = 0 			# time initialization

# filter boundary (Hz)
lowcut = 1
highcut = 10

start_position = 'mid_all' 

# define which part of fiber is used - not used when start_position is not 'mid_all'
start_channel = 56
end_channel = 430

ref_channel = 250		# reference point for correlation. currently used: 100, 200, 300, 400, 500, ... (depends on the part of the fiber that is being used)
nb_channel = 150		# number of channel used - not used when start_position is not 'mid_all'

Cross correlation

In [7]:
i = 1

for file in files :
	
	print(f'Processing file {i} out of {size_files}')
	
	if corr_type == 'phase-cross' :
		folder_destination = file.replace(path1, path2).replace(".nc", "/")

	elif corr_type == 'phase-auto' :
		folder_destination = file.replace(path1, path3).replace(".nc", "/")
		
		
	if not os.path.exists(folder_destination):
		os.mkdir(folder_destination)


	ds = xr.open_dataset(file)
	channels_coord = ds['channels']

	iDas_nc = xr.DataArray( 
		data=ds['strain_rates'],  # Remplacez par les données réelles de 'strain_rates'
		dims=("times", "channels"),  # Spécifiez les dimensions 
		coords={"times": ds['strain_rates'].coords["times"], "channels": channels_coord}
		)
    
	dt = ( iDas_nc['times'][1].item() - iDas_nc['times'][0].item() ) * 1e-9 # s
	fs = 1/dt # Hz

	f = open('error.txt','w') # files with few channels are not used and are stored into the file 'f'

	if iDas_nc.shape[1] < 463: 
		print(f'This file only has {iDas_nc.shape[1]} channels. Skipping this file')
		f.write(f'{iDas_nc.shape[1]} channels\n')
		# skipping file if the channel is not complefind . -type f -path "*/C*/" -exec mv {} DossierDestination \;te

	else :

		print(f'Filtering using bandpass filter between {lowcut} and {highcut} Hz ...')

		traces_filt = iDas_nc.copy()

    	# for j in [56,3249]:
		for j in range (start_channel,end_channel+1):
			try:
		    	#traces_filt[:,j] = filter_bandpass(traces_filt[:,j], lowcut, highcut, fs,order=4) # filter defined in stack_functions and using scipy.signal.butter filter
				traces_filt[:,j] = bandpass(traces_filt[:,j],lowcut,highcut,fs, zerophase=True) # obspy butterworth filter (faster)
			except:
				print('filtrage pas abouti pour j = ', j)
				#None
		
		length = iDas_nc.shape[0]	# total length of data

		# cross correlate

		if corr_type == 'phase-cross' :
			print('\nStarting the phase cross-correlation  ...')
			pcc, t = cross_correlate(traces_filt,folder_destination,window,lag,length,time,start_channel,end_channel,nb_channel,
                	                      ref_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)
			
		elif corr_type == 'phase-auto' :
			print('\nStarting the phase auto-correlation  ...')
			pcc_auto, t = auto_correlate(traces_filt,folder_destination,window,lag,length,time,start_channel,end_channel,nb_channel,
            	                          ref_channel,start_position,onebit=False,phase_cc=True,decimate=1,resample=1)


	f.close()
	i += 1

Processing file 1 out of 242
Filtering using bandpass filter between 1 and 10 Hz ...
no gauge length change

Starting the phase cross-correlation  ...
Start of the 1th time window. Starttime = 0.0 s
Start of the 2th time window. Starttime = 150.0 s
Start of the 3th time window. Starttime = 300.0 s
Start of the 4th time window. Starttime = 450.0 s
Start of the 5th time window. Starttime = 600.0 s
Start of the 6th time window. Starttime = 750.0 s
Start of the 7th time window. Starttime = 900.0 s
Start of the 8th time window. Starttime = 1050.0 s
Start of the 9th time window. Starttime = 1200.0 s
Start of the 10th time window. Starttime = 1350.0 s
Start of the 11th time window. Starttime = 1500.0 s
Start of the 12th time window. Starttime = 1650.0 s
Start of the 13th time window. Starttime = 1800.0 s
Start of the 14th time window. Starttime = 1950.0 s
Start of the 15th time window. Starttime = 2100.0 s
Start of the 16th time window. Starttime = 2250.0 s
Start of the 17th time window. Star

tf-PWS (time-frequency domain Phase Weighted Stack)

In [5]:
corr_type = 'phase-cross'

In [9]:
def empty_folder(folder):
    content = os.listdir(folder)
    return len(content) == 0

Classification correlograms by channel

In [7]:
if corr_type == 'phase-cross' :
    path = path2
elif corr_type == 'phase-auto' :
    path = path3

os.chdir(path)

for i_channel in range(start_channel, end_channel) :
    
    destination = path + f'all_correlograms_{ref_channel}_{i_channel}/'
    print(destination)
    
    if not os.path.exists(destination) : 
        os.mkdir(destination)

    os.system(f'find . -type f -wholename "*/Correlogram_{ref_channel}_{i_channel}_pcc/{ref_channel}_{i_channel}_*.sac" -exec cp {{}} {destination} \;')


/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_56/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_57/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_58/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_59/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_60/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_61/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_62/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_63/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_64/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_65/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_66/
/media/hugonnet/DISKENOIR/Correlograms/Cross-correlation/all_correlograms_250_67/
/media/hugonnet/

Calculation of tf-PWS

In [11]:
for i_channel in range(start_channel,end_channel+1):
    
    if corr_type == 'phase-cross' :
        folder = path2 + f'all_correlograms_{ref_channel}_{i_channel}/'

    elif corr_type == 'phase-auto' :
        folder = path3 + f'all_correlograms_{ref_channel}_{i_channel}/'
        
    os.system(f'cp {path4}ts_pws {folder}')
    os.chdir(folder)
    os.system('find . -type f -empty -print -delete') # delete empty file
    print(f'tf-pws for correlogram {ref_channel} and {i_channel}')
    os.system('ls --color=never *.sac > tf-pws.in')
    #os.system('ls *.sac > tf-pws.in')
    os.system('./ts_pws tf-pws.in osac="tspws_pcc"')
    # there will be a warning that says the header is corrupted, just ignore it since we just need the cross-correlogram time array, the header information is not important

tf-pws for correlogram 600 and 525

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 ReadData: Found the header corrupted when reading the 600_525_0_300_20220918_1735.sac file

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 wrsac: Found the header corrupted when writing the tl_tspws_pcc.sac file

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 net1=
 sta1=
 loc1=
 chn1=
 net2=
 sta2=
 loc2=
 chn2=
stla is not fine
stlo is not fine
stla is not fine
stlo is not fine
 wrsac: Found the header corrupted when writing the ts_pws_tspws_pcc.sac file
tf-pws for correlogram 600 and 526

 lat=-12345.000000
 lon=-12345.000000
 lat=-12345.000000
 lon=-12345.000000
 ne